In [2]:
# Set things up for later.
from numpy import *
import scipy.stats as stats
from scipy.optimize import minimize_scalar

# We set ourselves up for plots
import matplotlib.pyplot as plt
%matplotlib inline 

# Timing
from timeit import default_timer as timer

In [4]:
# See the notebook ES-Margins-Basics for derivation of these formulae
def ExpectedShortfallLogNormal(mu, sigma, lambd):
    es = -(1.0/lambd)*exp(mu+sigma*sigma*0.5)*stats.norm.cdf(stats.norm.ppf(lambd)-sigma)
    return es

def ExpectedShortfallNegativeLogNormal(mu, sigma, lambd):
    return (1.0/lambd)*exp(mu+sigma*sigma*0.5)*(1-stats.norm.cdf(stats.norm.ppf(1-lambd)-sigma))


# Let's see the margin in terms of P&L

def RiskFactorCalculator(sigma, lambd, tau, mu):
    """
    params to calculate risk Factor
    Returns a list of float numbers [riskFatorShort, riskFatorlong]
    """
    muBar = (mu-0.5*sigma*sigma)*tau
    sigmaBar = sqrt(tau)*sigma
    #print("PandL muBar %.5f and sigmaBar %.5f" % (muBar, sigmaBar))
    riskFactorShort = ExpectedShortfallNegativeLogNormal(muBar, sigmaBar, lambd)-1.0
    riskFactorLong = ExpectedShortfallLogNormal(muBar, sigmaBar, lambd)+1.0
    print("Is short bigger: %r" % (riskFactorShort >= riskFactorLong)) #testing if short is larger than long
    
    return [riskFactorShort, riskFactorLong]


r = 0.0; mu = 0.0; T = 1.0; sigma = 3.
tau = 1.0 / 365.25 / 24
lambd = 0.01

K_0 = 200
K_1 = 200

[riskFactorShort, riskFactorLong] = RiskFactorCalculator(sigma, lambd, tau, mu)

print("Risk factor short %.8f, long %.8f" % (riskFactorShort, riskFactorLong))
print("Risk factor short - long %.8f" % (riskFactorShort - riskFactorLong))


Is short bigger: True
Risk factor short 0.08864696, long 0.08227992
Risk factor short - long 0.00636704


## Expected shortfall for a Forward contract in Black-Scholes World
We consider a forward contract which allows the purchase of one unit of a (risky) asset at a future time $T>t$ for $K$ units of currency. 
We assume that time-value of money is given by a risk-free asset
$$dB_t = r B_t\,dt\,,\,\,\, B_0 = 1$$

The contract payoff (at time $T$) is $S_T - K$. 
The fair present value of this is $p_t = S_t - \frac{B_t}{B_T}K$. 
We consider a trader who sold such contract. 

While no model for risky asset is needed to price forward contracts (the price follows form no-arbitrage reasoning in a model-free way), we do need a model to assess the risk.
We assume that in the real world measure we have 
$$dS_t = \mu S_t\, dt + \sigma S_t \,dW_t\,,\,\,\,S_0 = S\,.$$



### One step
At time $t+\tau$ (with $\tau > 0$ small) we have $p_{t+\tau} = S_{t+\tau} - \frac{B_{t+\tau}}{B_T}K$.
Thus, due to cash invariance
$$\text{ES}_\lambda(-p_{t+\tau}) = \text{ES}_\lambda(-S_{t+\tau}) - \frac{B_{t+\tau}}{B_T}K\,.$$

We can solve the SDE for $(S_t)$ with Ito formula to get
$$S_{t+\tau} = S_t\exp\left((\mu-\frac12 \sigma^2)\tau + \sigma(W_{t+\tau}-W_t)\right)\,.$$
Recall that $W_{t+\tau}-W_t \sim \sqrt{\tau}N(0,1)$. 
Thus, with positive homogenienty of expected shortfall,
$$\text{ES}_\lambda(-p_{t+\tau}) = S_t \, \text{ES}_\lambda(-X) - \frac{B_{t+\tau}}{B_T}K\,,$$
where $X:=\exp(\bar \mu + \bar \sigma Z)$, with $Z\sim N(0,1)$ 


and with $\bar \mu := (\mu-\frac12 \sigma^2)\tau$,  $\bar \sigma := \sigma \sqrt{\tau}$.

# References
[Follmer, H. and Schied, A. - Convex and coherent risk measures, 2008](https://www.math.hu-berlin.de/~foellmer/papers/CCRM.pdf)

[Embrechts, P. and Wang, R. - Seven proofs for the subadditivity of expected shortfall, 2015](https://people.math.ethz.ch/~embrecht/ftp/Seven_Proofs.pdf)